# Installing Dependencies

In [4]:
using DelimitedFiles
using NearestNeighbors: KDTree, knn
using LinearAlgebra: norm

## Using General Arrays

## Using kNN

In [5]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X')
    end 

    # Loop over all cells to compute displacements
    index, _ = knn(kdtree, X[1,:], 14, true)
    for i in 2:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,:], 14, true)
        index = hcat(index, idxs)
    end
    return index
end

function force(X, r_max, s, K, NN)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs = NN[:,i]
        # Initialise variables
        global Xi = X[i,1:3]
        for j in idxs
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

force (generic function with 1 method)

In [8]:
R_Agg = 16

@time X = readdlm("../data/Init/Two_Sphere/$R_Agg.csv", ',', Float64, header=true)[1][:, 1:3]
r_max, s = 2, 1
K = 10
t, t_knn = 0, 10
dt = 0.5

  0.045041 seconds (565.10 k allocations: 15.185 MiB)


0.5

In [15]:
@time NN = NearNeighbor(X, t, t_knn)

  0.321361 seconds (36.27 k allocations: 1.922 GiB, 10.57% gc time)


14×6064 Matrix{Int64}:
   1    2    3    4    5    6    7  …  6059  6060  6061  6062  6063  6064
  25   26   27   40   42   43   44     6024  6025  6026  6038  6039  6040
  33   33   35   41   41   42   43     6023  6024  6025  6030  6031  6033
  32   34   34    9    9   10   11     6055  6057  6057  6031  6032  6032
   2    3    2    8   10   11    6     6056  6056  6060  6063  6064  6063
   4    1    6    5    4    5   35  …  6058  6061  6033  6059  6062  6061
   5    6    7   32    6    7    3     6060  6059  6064  6058  6060  6060
  24    5   26    1   33   34   51     6031  6032  6017  6037  6059  6039
  26   25   28   48    2    3   34     6062  6063  6034  6039  6038  6041
  41   27   43   33    1    2   36     6063  6064  6032  6023  6040  6025
 102   42  104   31   49   50  114  …  6015  6016  5954  5962  6024  5964
  20  103   22  111   34   35   52     6030  6033  6018  6045  5963  6047
  19   20   21   47   32   33   50     6032  6031  6016  6044  6045  6046
  42   21   44 

In [30]:
@time X_NN = [X[NN[:,i],:] for i in 1:1:size(X)[1]]

  0.071805 seconds (79.70 k allocations: 6.375 MiB, 92.69% compilation time)


6064-element Vector{Matrix{Float64}}:
 [-17.98 -2.89 -15.51; -17.98 -4.04 -13.88; … ; -18.98 -5.77 -13.88; -15.98 -0.58 -13.88]
 [-15.98 -2.89 -15.51; -15.98 -4.04 -13.88; … ; -16.98 -5.77 -13.88; -14.98 -5.77 -13.88]
 [-13.98 -2.89 -15.51; -13.98 -4.04 -13.88; … ; -14.98 -5.77 -13.88; -11.98 -0.58 -13.88]
 [-18.98 -1.15 -15.51; -19.98 -0.58 -13.88; … ; -20.98 1.15 -13.88; -16.98 1.15 -13.88]
 [-16.98 -1.15 -15.51; -15.98 -0.58 -13.88; … ; -18.98 -2.31 -13.88; -16.98 -1.15 -12.25]
 [-14.98 -1.15 -15.51; -13.98 -0.58 -13.88; … ; -16.98 -2.31 -13.88; -14.98 -1.15 -12.25]
 [-12.98 -1.15 -15.51; -11.98 -0.58 -13.88; … ; -14.98 1.15 -13.88; -15.98 -0.58 -13.88]
 [-19.98 0.58 -15.51; -18.98 1.15 -13.88; … ; -16.98 1.15 -13.88; -16.98 -1.15 -15.51]
 [-17.98 0.58 -15.51; -18.98 1.15 -13.88; … ; -15.98 -0.58 -13.88; -17.98 0.58 -12.25]
 [-15.98 0.58 -15.51; -14.98 1.15 -13.88; … ; -13.98 -0.58 -13.88; -15.98 0.58 -12.25]
 ⋮
 [16.98 -0.58 15.51; 15.98 -1.15 13.88; … ; 18.98 0.58 13.88; 16.98 -0.

In [ ]:
function force_2(X, r_max, s, K, NN)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    X_NN = [[X[NN[:,i],:]] for i in 1:1:size(X)[1]][m,:]
    # Scan neighbours
    # Initialise variables
    global Xi = X[i,:]
    for j in idxs
        global r = Xi - X[j,1:3]
        global dist = norm(r)
        # Calculate attraction/repulsion force differential here
        if dist < r_max
            global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
            dX[i,:] +=  r/dist * F
        end
    end
end